In [1]:
# Import necessary libraries
import asyncio
import sys
import os
import warnings
from devtools import pprint
# Add the src directory to the Python path and suppress warnings
sys.path.append(os.path.abspath('../src/agent'))
warnings.filterwarnings("ignore", category=FutureWarning)
from market_intelligence_agent.config import TEAM_MEMBERS
from langchain_core.messages import convert_to_messages
from langgraph_sdk import get_client
from datetime import datetime

import logging
import uuid


In [2]:
# LangGraph Client
lg_client = get_client(url="http://localhost:2024")

user_query = ["Is Tesla overvalued? Should I buy it now? do not use coder"]
user_input_messages = user_query

thread = await lg_client.threads.create()

print("Starting graph stream...")
# NOTE: The input dictionary is simplified
async for chunk in lg_client.runs.stream(
    thread["thread_id"],
    assistant_id="market_intelligence_agent",
    input={
        # Constants
        "TEAM_MEMBERS": TEAM_MEMBERS,
        # Runtime Variables
        "messages": user_input_messages,
        "current_timestamp": datetime.now(),
        "researcher_credits": 6,
        "market_credits": 6,
        "coder_credits": 0,
        "browser_credits": 3,
    },
    config={
        "recursion_limit": 150
    },
    stream_mode="values"
):
    messages = chunk.data.get("messages", None)
    if messages:
        pprint(convert_to_messages(messages)[-1])
    print("=" * 50)

print("Graph stream finished.")



Starting graph stream...
HumanMessage(
    content='Is Tesla overvalued? Should I buy it now?',
    additional_kwargs={},
    response_metadata={},
    id='b9bbf823-02cd-4d54-be37-15b3052015b2',
)
HumanMessage(
    content='Is Tesla overvalued? Should I buy it now?',
    additional_kwargs={},
    response_metadata={},
    id='b9bbf823-02cd-4d54-be37-15b3052015b2',
)
HumanMessage(
    content=(
        '{\n'
        '  "thought": "The user wants to know whether Tesla (TSLA) is currently over-valued and if it is a good time t'
        'o buy. A truly valuable answer will combine (1) hard valuation numbers, (2) historical context, (3) peer and '
        'market comparisons, (4) catalysts and risks, and (5) a forward-looking outlook. We therefore need recent fina'
        'ncial/market data, thorough quantitative analysis (multiples, DCF, return statistics), and a clear, well-stru'
        'ctured narrative that translates the data into an actionable view.",\n'
        '  "title": "Compreh

CancelledError: 

In [3]:
current_state = await lg_client.threads.get_state(thread["thread_id"])

In [4]:
current_state['values']['messages'][-1]['content']

'{"task": "Gather Tesla\'s recent financial statements and key metrics (revenue, net income, EPS, profit margins) for the past five years. Focus on accuracy and recency."}'

In [ ]:
from IPython.display import Markdown, display

def display_markdown(content):
    start_marker = "<response>\n"
    end_marker = "</response>\n"
    start_index = content.find(start_marker)
    end_index = content.find(end_marker)
    if start_index != -1 and end_index != -1:
        content = content[start_index + len(start_marker):end_index]
    return Markdown(content)
    
    
# Display the formatted markdown
display(display_markdown(current_state['values']['final_report']))

KeyError: 'final_report'

In [6]:
def save_string_to_md(text, directory, filename):
  if not os.path.exists(directory):
    os.makedirs(directory)

  # Create the full file path
  filepath = os.path.join(directory, filename + ".md")

  # Write the string to the file
  with open(filepath, "w") as f:
    f.write(text)

# Example usage:
output_directory = "../assets"
output_filename = "tesla_report"

save_string_to_md(current_state['values']['final_report'], output_directory, output_filename)
